# Sieci rekurencyjne

## Zadanie: Sieci rekurencyjne dla szeregów czasowych
Zbiór `jena_climate_2009_2016.csv` zawiera dane klimatyczne próbkowane co 10 minut.
1. Przygotuj dane do modelowania dzieląc zbiór na sekwencje składające się ze 120 pomiarów odpowiadających 5 dniom. Jako wartość docelową (`target`) ustaw temperaturę 24 godziny po końcu każdej sekwencji. Zastosuj metodę `timeseries_dataset_from_array()`. Skaluj dane i podziel je na zbiór treningowy, walidacyjny i testowy.
2. Stwórz i wytrenuj sieć składającą się z warstw LSTM. Na wyjściu zastosuj warstwę `Dense(1)` i `mse` jako funkcję błędu.
3. Zastosuj zbiór walidacyjny do porównania sieci o różnych architekturach. Testuj sieci z 1-2 warstwami LSTM o różnych wymiarach. Warstwy można regularyzować ustawiając parametr `recurrent_dropout`. Narysuj krzywe uczenia na danych treningowych i walidacyjnych. Następnie wykorzystaj zbiór testowy do ewaluacji wybranej sieci.

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

df = pd.read_csv("jena_climate_2009_2016.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'jena_climate_2009_2016.csv'

In [ ]:
X = df.drop(columns="Date Time").to_numpy()
y = df["T (degC)"].to_numpy()

N = X.shape[0]
train_size = int(0.5 * N)
val_size = int(0.25 * N)
test_size = N - train_size - val_size

scaler = StandardScaler()
scaler.fit(X[:train_size])
X = scaler.transform(X)

In [ ]:
sampling_rate = 6 # resample hourly
sequence_length = 120 # input sequences span 120 hours = 5 days
delay = sampling_rate * (sequence_length + 24 - 1) # targets delayed by 24h
batch_size = 256

train_data = tf.keras.utils.timeseries_dataset_from_array(
    X[:-delay],
    targets=y[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=0,
    end_index=train_size
)

val_data = tf.keras.utils.timeseries_dataset_from_array(
    X[:-delay],
    targets=y[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=train_size,
    end_index=(train_size + val_size)
)

test_data = tf.keras.utils.timeseries_dataset_from_array(
    X[:-delay],
    targets=y[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=train_size + val_size,
)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(sequence_length, X.shape[-1])),
    tf.keras.layers.LSTM(16),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer="adam",
              loss="mse",
              metrics=["mae"])

In [ ]:
model.fit(train_data,
          validation_data=val_data,
          epochs=5)

Epoch 1/5
819/819 ━━━━━━━━━━━━━━━━━━━━ 51s 61ms/step - loss: 81.3392 - mae: 6.8777 - val_loss: 17.4355 - val_mae: 3.1250
Epoch 2/5
819/819 ━━━━━━━━━━━━━━━━━━━━ 62s 76ms/step - loss: 15.9767 - mae: 3.0263 - val_loss: 11.5170 - val_mae: 2.6012
Epoch 3/5
819/819 ━━━━━━━━━━━━━━━━━━━━ 91s 111ms/step - loss: 11.4291 - mae: 2.6160 - val_loss: 10.1496 - val_mae: 2.4811
Epoch 4/5
819/819 ━━━━━━━━━━━━━━━━━━━━ 98s 120ms/step - loss: 10.4344 - mae: 2.5197 - val_loss: 9.3087 - val_mae: 2.3709
Epoch 5/5
819/819 ━━━━━━━━━━━━━━━━━━━━ 70s 86ms/step - loss: 9.7019 - mae: 2.4271 - val_loss: 9.3669 - val_mae: 2.3742


## Zadanie: Przetwarzanie tekstu sieciami rekurencyjnymi

1. Zbiór `aclImdb` zawiera przykłady pozytywnych (`pos`) i negatywnych (`neg`) recenzji filmów. Wczytaj zbiór za pomocą metody `text_dataset_from_directory()`. Dane są podzielone na zbiór treningowy i testowy - należy wczytać je osobno.
2. Stwórz warstwę `TextVectorization` z parametrem `output_modes='int'` i zastosuj metodę `adapt()` aby wygenerować słownik na podstawie zbioru treningowego.
3. Stwórz sieć LSTM i zastosuj ją do klasyfikacji recenzji. Bezpośrednio po warstwie wejściowej dodaj stworzoną wcześniej warstwę `TextVectorization`. Bezpośrednio po niej dodaj warstwę wektoryzującą słowa z tekstów. Może być to na przykład warstwa `Embedding`. Na wyjściu sieci dodaj warstwę `Dense(1, activation='sigmoid')`.
4. Porównaj działanie sieci rekurencyjnych o różnych architekturach. Zamiast warstw LSTM można stosować też warstwy GRU. Do regularyzaji warstw można stosować `recurrent_dropout`. Narysuj krzywe uczenia na danych treningowych i walidacyjnych.

In [ ]:
import tensorflow as tf

batch_size = 32

train_data = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)

valid_data = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
max_tokens = 5000
max_len = 600

text_vectorization = tf.keras.layers.TextVectorization(output_mode="int", max_tokens=max_tokens, output_sequence_length=max_len)
text_vectorization.adapt(train_data.map(lambda x, y: x))

2025-05-30 23:27:56.623348: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,), dtype="string"),
    text_vectorization,
    tf.keras.layers.Embedding(input_dim=max_tokens, output_dim=128, mask_zero=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

model.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 243s 309ms/step - accuracy: 0.7193 - loss: 0.5374 - val_accuracy: 0.8526 - val_loss: 0.3634
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 152s 195ms/step - accuracy: 0.8700 - loss: 0.3155 - val_accuracy: 0.8744 - val_loss: 0.2987
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 141s 180ms/step - accuracy: 0.9067 - loss: 0.2407 - val_accuracy: 0.8512 - val_loss: 0.3330
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 151s 193ms/step - accuracy: 0.9230 - loss: 0.1999 - val_accuracy: 0.8226 - val_loss: 0.5118
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 146s 187ms/step - accuracy: 0.8777 - loss: 0.3063 - val_accuracy: 0.8635 - val_loss: 0.3588
